# Task 1

In [1]:
import pandas as pd
from datetime import date
import concurrent.futures as futures
import multiprocessing

file_names = [
    "anagraficapazientiattivi",
    "diagnosi",
    "esamilaboratorioparametri",
    "esamilaboratorioparametricalcolati",
    "esamistrumentali",
    "prescrizionidiabetefarmaci",
    "prescrizionidiabetenonfarmaci",
    "prescrizioninondiabete",
]


def read_csv(filename):
    return pd.read_csv(filename, header=0, index_col=0)

In [2]:
# read all the dataset concurrently and store them in a dictionary with the name of the file as key
with futures.ThreadPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
    df_list = dict()
    for name in file_names:
        df_list[str(name)] = executor.submit(read_csv, f"../../sample/{name}.csv")

## Task 1.1
Select events of interest – we want only patients with at least one cardiovascular event in their trajectories.

In [3]:
df_diagnosi = df_list["diagnosi"].result()

"""
AMD047: Myocardial infarction
AMD048: Coronary angioplasty
AMD049: Coronary bypass
AMD071: Ictus
AMD081: Lower limb angioplasty
AMD082: Peripheral By-pass Lower Limbs
AMD208: Revascularization of intracranial and neck vessels
AMD303: Ischemic stroke
"""
AMD_OF_CARDIOVASCULAR_EVENT = [
    "AMD047",
    "AMD048",
    "AMD049",
    "AMD071",
    "AMD081",
    "AMD082",
    "AMD208",
    "AMD303",
]

In [4]:
print(
    "numero record presenti in diagnosi: ", len(df_diagnosi[["idana", "idcentro"]])
)  # 4427337
print(
    "numero pazienti presenti in diagnosi prima del punto 1: ",
    len(df_diagnosi[["idana", "idcentro"]].drop_duplicates()),
)  # 226303

numero record presenti in diagnosi:  4427337
numero pazienti unici presenti in diagnosi:  226303


In [5]:
# anagrafica table
df_anagrafica_attivi = df_list["anagraficapazientiattivi"].result()
# pd.read_csv("sample/anagraficapazientiattivi.csv", header=0, index_col=False)

df_anagrafica_attivi["annodiagnosidiabete"] = pd.to_datetime(
    df_anagrafica_attivi["annodiagnosidiabete"], format="%Y"
)
df_anagrafica_attivi["annonascita"] = pd.to_datetime(
    df_anagrafica_attivi["annonascita"], format="%Y"
)
df_anagrafica_attivi["annoprimoaccesso"] = pd.to_datetime(
    df_anagrafica_attivi["annoprimoaccesso"], format="%Y"
)
df_anagrafica_attivi["annodecesso"] = pd.to_datetime(
    df_anagrafica_attivi["annodecesso"], format="%Y"
)

In [6]:
print(
    "numero record presenti in anagrafica: ",
    len(df_anagrafica_attivi[["idana", "idcentro"]]),
)  # 250000
print(
    "numero pazienti presenti in anagrafica prima del punto 1: ",
    len(df_anagrafica_attivi[["idana", "idcentro"]].drop_duplicates()),
)  # 250000

print(
    "numero pazienti in anagrafica presenti in diagnosi:",
    len(
        df_anagrafica_attivi[["idana", "idcentro"]]
        .drop_duplicates()
        .merge(
            df_diagnosi[["idana", "idcentro"]].drop_duplicates(),
            how="inner",
            on=["idana", "idcentro"],
        )
    ),
)  # 226303

numero record presenti in anagrafica:  250000
numero pazienti unici in anagrafica:  250000
numero pazienti in anagrafica presenti in diagnosi: 226303


In [7]:
# Diagnosi relative a problemi cardiaci
df_diagnosi_problemi_cuore = df_diagnosi[
    df_diagnosi["codiceamd"].isin(AMD_OF_CARDIOVASCULAR_EVENT)
]

In [8]:
print(
    "numero pazienti presenti in diagnosi con codice amd in lista (con problemi al cuore): ",
    len(df_diagnosi_problemi_cuore[["idana", "idcentro"]].drop_duplicates()),
)  # 50000

numero pazienti presenti in diagnosi con codice amd in lista (con problemi al cuore):  50000


The following cell is part of the 1.6 point

In [9]:
# punto 6 fatto direttamente qui per alleggerire le computazioni
print(
    "numero pazienti presenti in diagnosi con codice amd in lista (con problemi al cuore) e con data presente: ",
    len(
        df_diagnosi_problemi_cuore[~df_diagnosi_problemi_cuore["data"].isna()][
            ["idana", "idcentro"]
        ].drop_duplicates()
    ),
)

# anagrafica pazienti con problemi al cuore, e relativa diagnosi
aa_prob_cuore = df_anagrafica_attivi.merge(
    df_diagnosi_problemi_cuore[~df_diagnosi_problemi_cuore["data"].isna()],
    on=["idcentro", "idana"],
    how="inner",
)

print(
    "numero pazienti presenti in anagrafica con problemi al cuore e data presente (dopo punto 1 e con 6): ",
    len(aa_prob_cuore[["idana", "idcentro"]].drop_duplicates()),
)

numero pazienti presenti in diagnosi con codice amd in lista (con problemi al cuore) e con data presente:  49997
numero pazienti presenti in anagrafica con problemi al cuore e data presente:  49997


In [ ]:
del df_diagnosi_problemi_cuore

## Task 1.2

Invalid feature cleaning - check for dates and time intervals [x, y] such that y < x and not x ≤ y. Check for years that do not make sense (e.g., events before the birth of a particular patient).

In [10]:
print(
    "numero righe con anno diagnosi diabete minore dell'anno di nascita: ",
    sum(aa_prob_cuore["annodiagnosidiabete"] < aa_prob_cuore["annonascita"]),
)  # 0

print(
    "numero righe con anno primo accesso minore dell'anno di nascita: ",
    sum(aa_prob_cuore["annoprimoaccesso"] < aa_prob_cuore["annonascita"]),
)  # 0

print(
    "numero righe con anno decesso minore dell'anno di nascita: ",
    sum(
        aa_prob_cuore["annodecesso"].fillna(pd.Timestamp.now())
        < aa_prob_cuore["annonascita"]
    ),
)  # 3 # 0 se seleziono solo quelli con casi rilevanti

# TODO: elimina, siccome non più rilevanti in quanto non presenti tra i pazienti con problemi al cuore
# aa_prob_cuore = aa_prob_cuore[
#     aa_prob_cuore["annodecesso"].fillna(pd.Timestamp.now())
#     >= aa_prob_cuore["annonascita"]
# ]

# print(
#     "numero pazienti dopo scarto: ",
#     len(aa_prob_cuore[["idana", "idcentro"]].drop_duplicates()),
# )

# print(
#     "numero righe dopo scarto con anno decesso minore dell'anno di nascita: ",
#     sum(
#         aa_prob_cuore["annodecesso"].fillna(pd.Timestamp.now())
#         < aa_prob_cuore["annonascita"]
#     ),
# )

print(
    "numero pazienti con anno decesso maggiore dell'anno 2022: ",
    sum(
        aa_prob_cuore["annodecesso"] > pd.to_datetime(2023, format="%Y"),
    ),
)  # 0

# the conversion in datetime don't work for the year 0001 or 0000
# print(
#     "numero pazienti con anno di nascita negativo: ",
#     sum(
#         aa_prob_cuore["annonascita"] < pd.to_datetime("0001", format="%Y"),
#     ),
# )

print(
    "numero righe con anno primo accesso a N/A: ",
    sum(aa_prob_cuore["annoprimoaccesso"].isna()),
)  # 25571

print(
    "numero righe con anno diagnosi diabete a N/A: ",
    sum(aa_prob_cuore["annodiagnosidiabete"].isna()),
)  # 2234

print(
    "numero righe con anno primo accesso maggiore dell'anno decesso: ",
    sum(
        aa_prob_cuore["annoprimoaccesso"]
        > aa_prob_cuore["annodecesso"].fillna(pd.Timestamp.now())
    ),
)  # 34 (dopo scarto precedente 33)
# solo pazienti con eventi cardiovascolari: 14

numero righe con anno diagnosi diabete minore dell'anno di nascita:  0
numero righe con anno primo accesso minore dell'anno di nascita:  0
numero righe con anno decesso minore dell'anno di nascita:  0
numero pazienti con anno decesso maggiore dell'anno 2022:  0
numero righe con anno primo accesso a N/A:  25571
numero righe con anno diagnosi diabete a N/A:  2234
numero righe con anno primo accesso maggiore dell'anno decesso:  14


In [11]:
aa_prob_cuore = aa_prob_cuore[
    (
        aa_prob_cuore["annoprimoaccesso"]
        <= aa_prob_cuore["annodecesso"].fillna(pd.Timestamp.now())
    )
    | (aa_prob_cuore["annoprimoaccesso"].isna())
]

print(
    "numero righe dopo scarto con anno primo accesso maggiore dell'anno decesso: ",
    sum(
        aa_prob_cuore["annoprimoaccesso"]
        > aa_prob_cuore["annodecesso"].fillna(pd.Timestamp.now())
    ),
)

print(
    "numero pazienti dopo scarto: ",
    len(aa_prob_cuore[["idana", "idcentro"]].drop_duplicates()),
)

numero righe dopo scarto con anno primo accesso maggiore dell'anno decesso:  0
numero pazienti dopo scarto:  49990


In [12]:
# print("tipi possibili di diabete: ", aa_prob_cuore["tipodiabete"].unique())
# in anagrafica abbiamo solo pazienti con diagnosi di diabete di tipo 2 valore 5 in 'tipodiabete'
# quindi possiamo fillare l'annodiagnosidiabete con l'annoprimoaccesso

# visto che il tipo diabete è sempre lo stesso si può eliminare la colonna dal df per risparmiare memoria
aa_prob_cuore.drop(columns=["tipodiabete"], inplace=True)

In [13]:
print(
    "numero righe con anno diagnosi diabete maggiore dell'anno decesso: ",
    sum(
        aa_prob_cuore["annodiagnosidiabete"]
        > aa_prob_cuore["annodecesso"].fillna(pd.Timestamp.now())
    ),
)  # 22 (dopo scarto precedente 2)
# 14 solo paziente con eventi cadriovascolari

aa_prob_cuore = aa_prob_cuore[
    (
        aa_prob_cuore["annodiagnosidiabete"]
        <= aa_prob_cuore["annodecesso"].fillna(pd.Timestamp.now())
    )
    | aa_prob_cuore["annodiagnosidiabete"].isna()
]

print(
    "numero righe dopo scarto con anno diagnosi diabete maggiore dell'anno decesso: ",
    sum(
        aa_prob_cuore["annodiagnosidiabete"]
        > aa_prob_cuore["annodecesso"].fillna(pd.Timestamp.now())
    ),
)

print(
    "numero pazienti dopo scarto: ",
    len(aa_prob_cuore[["idana", "idcentro"]].drop_duplicates()),
)

print(
    "numero righe con anno diagnosi diabete a N/A, ma che hanno l'anno di primo accesso: ",
    len(
        aa_prob_cuore[
            aa_prob_cuore["annodiagnosidiabete"].isna()
            & aa_prob_cuore["annoprimoaccesso"].notnull()
        ][["idana", "idcentro"]]
    ),
)  # 1797
# TODO con questa info si potrebbe fillare l'annodiagnosidiabete con l'annoprimoaccesso

numero righe con anno diagnosi diabete maggiore dell'anno decesso:  3
numero righe dopo scarto con anno diagnosi diabete maggiore dell'anno decesso:  0
numero pazienti dopo scarto:  49987
numero righe con anno diagnosi diabete a N/A, ma che hanno l'anno di primo accesso:  1797


In [14]:
# questi son tutti a 0
print(
    "numero righe anno diagnosi diabete minore anno di nascita: ",
    sum(aa_prob_cuore["annodiagnosidiabete"] < aa_prob_cuore["annonascita"]),
)  # 0
print(
    "numero righe anno primo accesso minore anno di nascita: ",
    sum(aa_prob_cuore["annoprimoaccesso"] < aa_prob_cuore["annonascita"]),
)  # 0
print(
    "numero righe anno decesso minore anno di nascita: ",
    sum(aa_prob_cuore["annodecesso"] < aa_prob_cuore["annonascita"]),
)  # 0
# print(sum(aa_prob_cuore["annodecesso"] > 2022))  # 0
# print(sum(aa_prob_cuore["annonascita"] < 0))  # 0

numero righe anno diagnosi diabete minore anno di nascita:  0
numero righe anno primo accesso minore anno di nascita:  0
numero righe anno decesso minore anno di nascita:  0


In [15]:
# 7 pazienti hanno la data di primo accesso maggiore della data di decesso -> da scartare
# i 7 non sono presenti tra i pazienti con eventi cardiovascolari
print(
    "numero righe con data di primo accesso maggiore della data di decesso: ",
    sum(aa_prob_cuore["annoprimoaccesso"] > aa_prob_cuore["annodecesso"]),
)  # 14 righe di cui 7 unici # 0 tra i pazienti con eventi cardiovascolari

print(
    "numero pazienti unici con data di primo accesso maggiore della data di decesso: ",
    len(
        aa_prob_cuore[aa_prob_cuore["annoprimoaccesso"] > aa_prob_cuore["annodecesso"]][
            ["idana", "idcentro"]
        ].drop_duplicates()
    ),
)

# 5 pazienti hanno la data di diagnosi di diabete maggiore della data di decesso -> da scartare
# i 5 non sono presenti tra i pazienti con eventi cardiovascolari
print(
    "numero righe con data di diagnosi di diabete maggiore della data di decesso: ",
    sum(aa_prob_cuore["annodiagnosidiabete"] > aa_prob_cuore["annodecesso"]),
)  # 9 righe di cui 5 unici # 0 tra i pazienti con eventi cardiovascolari

print(
    "numero pazienti unici con data di diagnosi di diabete maggiore della data di decesso: ",
    len(
        aa_prob_cuore[
            aa_prob_cuore["annodiagnosidiabete"] > aa_prob_cuore["annodecesso"]
        ][["idana", "idcentro"]].drop_duplicates()
    ),
)

print(
    "numero righe con anno diagnosi diabete a N/A: ",
    sum(aa_prob_cuore["annodiagnosidiabete"].isna()),
)  # 2234

print(
    "numero pazienti unici con anno diagnosi diabete a N/A: ",
    len(
        aa_prob_cuore[aa_prob_cuore["annodiagnosidiabete"].isna()][
            ["idana", "idcentro"]
        ].drop_duplicates()
    ),
)  # 526

# in anagrafica abbiamo solo pazienti con diagnosi di diabete di tipo 2 valore 5 in 'tipodiabete'
# quindi possiamo fillare l'annodiagnosidiabete con l'annoprimoaccesso
print(
    "numero righe con anno diagnosi diabete a N/A ma con anno primo accesso presente: ",
    len(
        aa_prob_cuore[
            aa_prob_cuore["annodiagnosidiabete"].isna()
            & aa_prob_cuore["annoprimoaccesso"].notnull()
        ][["idana", "idcentro"]]
    ),
)  # 1797

print(
    "numero pazienti unici con anno diagnosi diabete a N/A ma con anno primo accesso presente: ",
    len(
        aa_prob_cuore[
            aa_prob_cuore["annodiagnosidiabete"].isna()
            & aa_prob_cuore["annoprimoaccesso"].notnull()
        ][["idana", "idcentro"]].drop_duplicates()
    ),
)  # 365

numero righe con data di primo accesso maggiore della data di decesso:  0
numero pazienti unici con data di primo accesso maggiore della data di decesso:  0
numero righe con data di diagnosi di diabete maggiore della data di decesso:  0
numero pazienti unici con data di diagnosi di diabete maggiore della data di decesso:  0
numero righe con anno diagnosi diabete a N/A:  2234
numero pazienti unici con anno diagnosi diabete a N/A:  526
numero righe con anno diagnosi diabete a N/A ma con anno primo accesso presente:  1797
numero pazienti unici con anno diagnosi diabete a N/A ma con anno primo accesso presente:  365


The following cell is about point 1.6

In [16]:
# fill annodiagnosidiabete with annoprimoaccesso where aa_prob_cuore["annodiagnosidiabete"].isna() & aa_prob_cuore["annoprimoaccesso"].notnull()
# for POINT 6
aa_prob_cuore.loc[
    aa_prob_cuore["annodiagnosidiabete"].isna()
    & aa_prob_cuore["annoprimoaccesso"].notnull(),
    "annodiagnosidiabete",
] = aa_prob_cuore.loc[
    aa_prob_cuore["annodiagnosidiabete"].isna()
    & aa_prob_cuore["annoprimoaccesso"].notnull(),
    "annoprimoaccesso",
]

print(
    "numero pazienti unici dopo fill con anno diagnosi diabete a N/A ma con anno primo accesso presente: ",
    len(
        aa_prob_cuore[
            aa_prob_cuore["annodiagnosidiabete"].isna()
            & aa_prob_cuore["annoprimoaccesso"].notnull()
        ][["idana", "idcentro"]].drop_duplicates()
    ),
)

print(
    "numero righe dopo fill con anno diagnosi diabete a N/A: ",
    sum(aa_prob_cuore["annodiagnosidiabete"].isna()),
)  # 437

print(
    "numero pazienti unici dopo fill con anno diagnosi diabete a N/A: ",
    len(
        aa_prob_cuore[aa_prob_cuore["annodiagnosidiabete"].isna()][
            ["idana", "idcentro"]
        ].drop_duplicates()
    ),
)  # 161

print(
    "numero righe dopo fill con anno primo accesso a N/A: ",
    sum(aa_prob_cuore["annoprimoaccesso"].isna()),
)  #

print(
    "numero pazienti unici dopo fill con anno primo accesso a N/A: ",
    len(
        aa_prob_cuore[aa_prob_cuore["annoprimoaccesso"].isna()][
            ["idana", "idcentro"]
        ].drop_duplicates()
    ),
)

numero pazienti unici dopo fill con anno diagnosi diabete a N/A ma con anno primo accesso presente:  0
numero righe dopo fill con anno diagnosi diabete a N/A:  437
numero pazienti unici dopo fill con anno diagnosi diabete a N/A:  161
numero righe dopo fill con anno primo accesso a N/A:  25569
numero pazienti unici dopo fill con anno primo accesso a N/A:  6970


In [17]:
print(len(aa_prob_cuore[["idana", "idcentro"]].drop_duplicates()))

print(aa_prob_cuore.isna().sum())

49987
idcentro                    0
idana                       0
sesso                       0
annodiagnosidiabete       437
scolarita              150789
statocivile            106756
professione            111335
origine                230207
annonascita                 0
annoprimoaccesso        25569
annodecesso            207593
data                        0
codiceamd                   0
valore                      0
dtype: int64


In [18]:
def printSexInfo(dataset):
    dataset = dataset[["idcentro", "idana", "sesso"]].drop_duplicates()
    print("numero righe del df: ", len(dataset))

    print("Sex info")
    print(dataset["sesso"].unique())
    print("sesso ad N/A", dataset["sesso"].isna().sum())
    print("Maschi: ", sum(dataset["sesso"].isin(["M"])))
    print("Femmine: ", sum(dataset["sesso"].isin(["F"])))

In [19]:

def getDeaseasePercentage(dataset, deaseases):
    print("Deasease: ", deaseases)
    # print(dataset.columns)
    percent = "Percentage of deasease:\n"
    dataset = dataset[["idcentro", "idana", "codiceamd"]].drop_duplicates()
    print("numero righe del df: ", len(dataset))

    for deasease in deaseases:
        # print("Deasease: ", deasease)
        tempdataset = dataset[dataset["codiceamd"].isin([deasease])]
        tempdataset2 = tempdataset[["idana", "idcentro"]].drop_duplicates()
        total = len(dataset[["idana", "idcentro"]].drop_duplicates())
        percent += (
            deasease
            + ": "
            + str(len(tempdataset2) / total * 100)
            + "%\t"
            + str(len(tempdataset2))
            + " su "
            + str(total)
            + "\n"
        )
    print(percent)

In [20]:

def getInfoOnDiagnosi(df):
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("Info on diagnosi")
    print(
        "il dizionario stampato è formattato così: 'chiave': [minori, uguali, maggiori] rispetto a data"
    )
    dates = ["annodiagnosidiabete", "annonascita", "annoprimoaccesso", "annodecesso"]
    stampami = dict()
    df = df[
        [
            "idcentro",
            "idana",
            "annodiagnosidiabete",
            "annonascita",
            "annoprimoaccesso",
            "annodecesso",
            "data",
        ]
    ].drop_duplicates()

    print("numero righe del df: ", len(df))

    for d in dates:
        # where the nan values goes? they are counted as what? minor or major?
        minor = (df[d] < df["data"]).sum()
        equal = (df[d] == df["data"]).sum()
        more = (df[d] > df["data"]).sum()
        stampami[d] = [minor, equal, more]

    print(stampami)

In [21]:
getInfoOnDiagnosi(aa_prob_cuore)
# Sesso
print("In anagrafica attivi abbiamo:")
printSexInfo(aa_prob_cuore)
print("Fra i pazienti con problemi al cuore abbiamo:")
printSexInfo(aa_prob_cuore)
# Deasease Distribution
getDeaseasePercentage(aa_prob_cuore, AMD_OF_CARDIOVASCULAR_EVENT)
# TODO: qui i numeri non tornano quindi significa che stessi pazienti hanno avuto più codici amd diversi
# ora vai a capire in ambito medico se significa che hanno più problemi diversi o che hanno avuto diverse diagnosi,
# che la malattia progredisce e quindi cambia codice amd, bho
# provo a capire quali sono i pazienti che hanno avuto più codici amd diversi:
print(
    "numero pazienti con più codici amd diversi: ",
    len(
        aa_prob_cuore[aa_prob_cuore[["idana", "idcentro"]].duplicated(keep=False)][
            ["idana", "idcentro"]
        ].drop_duplicates()
    ),
)

# print(
#     "pazienti con più codici amd diversi: ",
#     aa_prob_cuore[aa_prob_cuore[["idana", "idcentro"]].duplicated(keep=False)][["idana", "idcentro"]].drop_duplicates(),
# )

print(
    "numero pazienti con un unico codice amd: ",
    len(
        aa_prob_cuore[~aa_prob_cuore[["idana", "idcentro"]].duplicated(keep=False)][
            ["idana", "idcentro"]
        ].drop_duplicates()
    ),
)

# TODO: qui si potrebbe calcolare anche qual'è la percentuale in base al sesso e casomai anche per età

# TODO: qui si potrebbe pensare di controllare se l'anno di nascita è uguale all' anno decesso e la data (del controllo?)
# è maggiore dell'anno primo accesso e di diagnosi del diabete di settare a nan l'anno di decesso in modo da non dover
# eliminare quei dati (però chi ti dice che è il decesso l'errore e non le visite?)

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Info on diagnosi
il dizionario stampato è formattato così: 'chiave': [minori, uguali, maggiori] rispetto a data
numero righe del df:  180684
{'annodiagnosidiabete': [163737, 548, 16023], 'annonascita': [175118, 0, 5566], 'annoprimoaccesso': [134286, 246, 25984], 'annodecesso': [992, 6, 18776]}
In anagrafica attivi abbiamo:
numero righe del df:  49987
Sex info
['M' 'F']
sesso ad N/A 0
Maschi:  36259
Femmine:  13728
Fra i pazienti con problemi al cuore abbiamo:
numero righe del df:  49987
Sex info
['M' 'F']
sesso ad N/A 0
Maschi:  36259
Femmine:  13728
Deasease:  ['AMD047', 'AMD048', 'AMD049', 'AMD071', 'AMD081', 'AMD082', 'AMD208', 'AMD303']
numero righe del df:  70922
Percentage of deasease:
AMD047: 52.59167383519715%	26289 su 49987
AMD048: 37.603776982015326%	18797 su 49987
AMD049: 21.22951967511553%	10612 su 49987
AMD071: 8.79828755476424%	4398 su 49987
AMD081: 7.671994718626843%	3835 su 49987
AMD082: 3.11280

In [22]:
aa_prob_cuore = aa_prob_cuore.drop(
    aa_prob_cuore[
        aa_prob_cuore["annoprimoaccesso"] > aa_prob_cuore["annodecesso"]
    ].index,
)
print(len(aa_prob_cuore[["idana", "idcentro"]].drop_duplicates()))

49987


In [23]:
# TODO: qui si potrebbe pensare di controllare se l'anno di nascita è uguale all' anno decesso e la data (del controllo?)
# è maggiore dell'anno primo accesso e di diagnosi del diabete di settare a nan l'anno di decesso in modo da non dover
# eliminare quei dati (però chi ti dice che è il decesso l'errore e non le visite?)

Cancellabile la prossima cella? 

In [24]:
aa_prob_cuore = aa_prob_cuore.drop(
    aa_prob_cuore[
        aa_prob_cuore["annodiagnosidiabete"] > aa_prob_cuore["annodecesso"]
    ].index,
)  # già fatto sopra? ops

print("dopo scarto :")
print(len(aa_prob_cuore[["idana", "idcentro"]].drop_duplicates()))

dopo scarto :
49987


In [25]:
# siccome più della metà dei pazienti che hanno problemi al cuore
# hanno l'anno di diagnosi di diabete minore dell'anno di primo accesso
# noi abbiamo deciso di fillare l'annodiagnosidiabete con l'anno primo accesso
print(
    "numero pazienti unici con anno diagnosi diabete minore dell'anno primo accesso: ",
    len(
        aa_prob_cuore[
            aa_prob_cuore["annodiagnosidiabete"] < aa_prob_cuore["annoprimoaccesso"]
        ][["idana", "idcentro"]].drop_duplicates()
    ),
)  # 27592

print(
    "numero pazienti unici con anno diagnosi diabete maggiore dell'anno primo accesso: ",
    len(
        aa_prob_cuore[
            aa_prob_cuore["annodiagnosidiabete"] >= aa_prob_cuore["annoprimoaccesso"]
        ][["idana", "idcentro"]].drop_duplicates()
    ),
)  # 15426

numero pazienti unici con anno diagnosi diabete minore dell'anno primo accesso:  27591
numero pazienti unici con anno diagnosi diabete maggiore dell'anno primo accesso:  15426


In [26]:
print("All filtered :")
aa_prob_cuore = aa_prob_cuore.dropna(subset="annodiagnosidiabete")
print(len(aa_prob_cuore[["idana", "idcentro"]].drop_duplicates()))  # 49829

### Punto 2 per dataset diversi da anagrafica attivi e diagnosi

## Carica i dataset
print("############## LOADING DATASETS ##############")

df_esami_par = df_list["esamilaboratorioparametri"].result()
print("loaded esami parametri")

df_esami_par_cal = df_list["esamilaboratorioparametricalcolati"].result()
print("loaded esami parametri calcolati")

df_esami_stru = df_list["esamistrumentali"].result()
print("loaded esami strumentali")

df_diagnosi = df_list["diagnosi"].result()
print("loaded diagnosi")

df_prescrizioni_diabete_farmaci = df_list["prescrizionidiabetefarmaci"].result()
print("loaded prescrizioni diabete farmaci")

df_prescrizioni_diabete_non_farmaci = df_list["prescrizionidiabetenonfarmaci"].result()
print("loaded prescrizioni diabete non farmaci")

df_prescirizioni_non_diabete = df_list["prescrizioninondiabete"].result()
print("loaded prescrizioni non diabete")

All filtered :
49826
############## LOADING DATASETS ##############
loaded esami parametri
loaded esami parametri calcolati
loaded esami strumentali
loaded diagnosi
loaded prescrizioni diabete farmaci
loaded prescrizioni diabete non farmaci
loaded prescrizioni non diabete


In [27]:
del df_list

In [28]:
## Calcola le chiavi dei pazienti di interesse
aa_cuore_dates = aa_prob_cuore[
    [
        "idana",
        "idcentro",
        "annonascita",
        "annoprimoaccesso",
        "annodecesso",
    ]
].drop_duplicates()
print(len(aa_cuore_dates))

49826


In [29]:
## Cast string to datatime
def cast_to_datetime(df, col, format="%Y-%m-%d"):
    df[col] = pd.to_datetime(df[col], format=format)
    return df[col]

In [30]:
df_esami_par = df_esami_par.merge(aa_cuore_dates, on=["idana", "idcentro"], how="inner")
print(
    "numero pazienti esami par: ",
    len(df_esami_par[["idana", "idcentro"]].drop_duplicates()),
)

df_esami_par_cal = df_esami_par_cal.merge(
    aa_cuore_dates, on=["idana", "idcentro"], how="inner"
)
print(
    "numero pazienti esami par cal: ",
    len(df_esami_par_cal[["idana", "idcentro"]].drop_duplicates()),
)

df_esami_stru = df_esami_stru.merge(
    aa_cuore_dates, on=["idana", "idcentro"], how="inner"
)
print(
    "numero pazienti esami stru: ",
    len(df_esami_stru[["idana", "idcentro"]].drop_duplicates()),
)


df_diagnosi = df_diagnosi.merge(aa_cuore_dates, on=["idana", "idcentro"], how="inner")
print(
    "numero pazienti diagnosi: ",
    len(df_diagnosi[["idana", "idcentro"]].drop_duplicates()),
)

df_prescrizioni_diabete_farmaci = df_prescrizioni_diabete_farmaci.merge(
    aa_cuore_dates, on=["idana", "idcentro"], how="inner"
)
print(
    "numero pazienti prescrizioni diabete farmaci: ",
    len(df_prescrizioni_diabete_farmaci[["idana", "idcentro"]].drop_duplicates()),
)


df_prescrizioni_diabete_non_farmaci = df_prescrizioni_diabete_non_farmaci.merge(
    aa_cuore_dates, on=["idana", "idcentro"], how="inner"
)
print(
    "numero pazienti prescrizioni diabete non farmaci: ",
    len(df_prescrizioni_diabete_non_farmaci[["idana", "idcentro"]].drop_duplicates()),
)

df_prescirizioni_non_diabete = df_prescirizioni_non_diabete.merge(
    aa_cuore_dates, on=["idana", "idcentro"], how="inner"
)
print(
    "numero pazienti prescrizioni non diabete: ",
    len(df_prescirizioni_non_diabete[["idana", "idcentro"]].drop_duplicates()),
)

numero pazienti esami par:  49558
numero pazienti esami par cal:  49270
numero pazienti esami stru:  37080
numero pazienti diagnosi:  49826
numero pazienti prescrizioni diabete farmaci:  46398
numero pazienti prescrizioni diabete non farmaci:  27671
numero pazienti prescrizioni non diabete:  46005


In [31]:
del aa_cuore_dates

In [32]:
list_of_df = [
    df_esami_par,
    df_esami_par_cal,
    df_esami_stru,
    df_diagnosi,
    df_prescrizioni_diabete_farmaci,
    df_prescrizioni_diabete_non_farmaci,
    df_prescirizioni_non_diabete,
]

## Cast string to datetime
for df in list_of_df:
    df["data"] = cast_to_datetime(df, "data", format="%Y-%m-%d")

In [33]:
def clean_between_dates(df, col="data", col_start="annonascita", col_end="annodecesso"):
    # this create a temporary df with only the patients of interest
    df1 = df.merge(aa_cuore_dates, on=["idana", "idcentro"], how="inner")
    print(
        "numero pazienti: ",
        len(df1[["idana", "idcentro"]].drop_duplicates()),
    )
    # non conosco la data in cui il dataset è stato samplato quindi ho usato il timestamp corrente(adesso) come workaround.
    df1 = df1[
        (df1[col] >= df1[col_start])
        & (df1[col] <= df1[col_end].fillna(pd.Timestamp.now()))
    ]
    # this ensure that the columns of patients of interest are the same as the original
    # df filtered only by the keys of the processed patients
    df = df.merge(
        df1[["idana", "idcentro"]].drop_duplicates(), on=["idana", "idcentro"]
    )
    return df

In [34]:
for df in list_of_df:
    df = clean_between_dates(df)

## Task 1.3

In [35]:
# TODO: verify if the concat is correct as the same as merge, and also if is the best way to do this
# TODO: here we must use also prescrizioni? or only esami and diagnosi? I think for me has more sense to only look
#  at esami and diagnosi (and also the specific tal about examinations and diagnosis and not about prescrizioni),
#  but I'm not 100% sure

In [36]:
df_diagnosi_and_esami = pd.concat(
    objs=(
        idf.set_index(["idana", "idcentro"])
        for idf in [
            df_diagnosi[["idana", "idcentro", "data"]],
            df_esami_par[["idana", "idcentro", "data"]],
            df_esami_par_cal[["idana", "idcentro", "data"]],
            df_esami_stru[["idana", "idcentro", "data"]],
        ]
    ),
    # ignore_index=True,
    join="inner",
).reset_index()  # 49768

In [37]:
print(len(df_diagnosi_and_esami[["idana", "idcentro"]].drop_duplicates()))

print(
    "numero pazienti di interesse inizio punto 3: ",
    len(df_diagnosi_and_esami[["idana", "idcentro"]].drop_duplicates()),
)
# print(df_diagnosi_and_esami.head())
# print(df_diagnosi_and_esami.info())

49826
numero pazienti di interesse inizio punto 3:  49826


In [38]:
groups_diagnosi_and_esami = df_diagnosi_and_esami.groupby(["idana", "idcentro"]).agg(
    {"data": ["min", "max"]}
)

groups_diagnosi_and_esami["data_min"] = groups_diagnosi_and_esami["data"]["min"]
groups_diagnosi_and_esami["data_max"] = groups_diagnosi_and_esami["data"]["max"]
# print(groups_diagnosi_and_esami.head(30))
print("groups_diagnosi_and_esami")

groups_diagnosi_and_esami["diff"] = (
    groups_diagnosi_and_esami["data_max"] - groups_diagnosi_and_esami["data_min"]
)

# print(groups_diagnosi_and_esami.head(30))
print(
    "numero di pazienti con tutte le date in un unico giorno: ",
    len(
        groups_diagnosi_and_esami[
            groups_diagnosi_and_esami["diff"] == pd.Timedelta("0 days")
        ]
    ),
)
print(
    "numero di pazienti con tutte le date in un unico mese: ",
    len(
        groups_diagnosi_and_esami[
            groups_diagnosi_and_esami["diff"] < pd.Timedelta("31 days")
        ]
    ),
)

groups_diagnosi_and_esami = groups_diagnosi_and_esami[
    groups_diagnosi_and_esami["diff"] >= pd.Timedelta("31 days")
]
groups_diagnosi_and_esami = groups_diagnosi_and_esami.sort_values(by=["diff"])
print("paziente traiettoria minima: ", groups_diagnosi_and_esami.head(2))
print("paziente traiettoria massima: ", groups_diagnosi_and_esami.tail(4))
print(
    "numero pazienti fine punto 3: ",
    len(groups_diagnosi_and_esami),
)

groups_diagnosi_and_esami
numero di pazienti con tutte le date in un unico giorno:  535
numero di pazienti con tutte le date in un unico mese:  871
paziente traiettoria minima:                       data              data_min   data_max    diff
                      min        max                              
idana idcentro                                                    
3355  100      2014-09-08 2014-10-09 2014-09-08 2014-10-09 31 days
7622  154      2007-05-05 2007-06-05 2007-05-05 2007-06-05 31 days
paziente traiettoria massima:                       data              data_min   data_max       diff
                      min        max                                 
idana idcentro                                                       
3053  142      1900-01-02 2019-08-19 1900-01-02 2019-08-19 43693 days
221   82       1900-01-20 2019-09-11 1900-01-20 2019-09-11 43698 days
729   14       1900-01-07 2019-09-05 1900-01-07 2019-09-05 43705 days
3273  23       1900-01-05 2019-09-23

In [39]:
groups_diagnosi_and_esami_keys = (
    groups_diagnosi_and_esami.stack()
    .reset_index()[["idana", "idcentro"]]
    .drop_duplicates()
)

print(len(groups_diagnosi_and_esami_keys))

48955


In [40]:
del groups_diagnosi_and_esami

## Task 1.4

In [41]:
print("prima update: ")
amd004 = df_esami_par[df_esami_par["codiceamd"] == "AMD004"]["valore"]
print("numero AMD004 minori di 40: ", len(amd004[amd004.astype(float) < 40]))
print("numero AMD004 maggiori di 200: ", len(amd004[amd004.astype(float) > 200]))

# df_esami_par_copy = df_esami_par.copy()
mask = df_esami_par["codiceamd"] == "AMD004"
df_esami_par.loc[mask, "valore"] = df_esami_par.loc[mask, "valore"].clip(40, 200)
# would like to use this single line but from documentation it seems that it can cause problems
# so we must use this in two lines with a precomputation of a mask
# df_esami_par["valore"].update(
#     df_esami_par[df_esami_par["codiceamd"] == "AMD004"]["valore"].clip(40, 200)
# )

mask = df_esami_par["codiceamd"] == "AMD005"
df_esami_par.loc[mask, "valore"] = df_esami_par.loc[mask, "valore"].clip(40, 130)

mask = df_esami_par["codiceamd"] == "AMD007"
df_esami_par.loc[mask, "valore"] = df_esami_par.loc[mask, "valore"].clip(50, 500)

mask = df_esami_par["codiceamd"] == "AMD008"
df_esami_par.loc[mask, "valore"] = df_esami_par.loc[mask, "valore"].clip(5, 15)

print("dopo update: ")
amd004_dopo = df_esami_par[df_esami_par["codiceamd"] == "AMD004"]["valore"]

print("numero AMD004 minori di 40 dopo filtro: ", len(amd004_dopo[amd004_dopo < 40]))
print(
    "numero AMD004 maggiori di 200 dopo filtro: ",
    len(amd004_dopo[amd004_dopo.astype(float) > 200]),
)

prima update: 
numero AMD004 minori di 40:  462
numero AMD004 maggiori di 200:  967
dopo update: 
numero AMD004 minori di 40 dopo filtro:  0
numero AMD004 maggiori di 200 dopo filtro:  0


In [42]:
print("prima update: ")

stitch002 = df_esami_par_cal[df_esami_par_cal["codicestitch"] == "STITCH002"]["valore"]
print("numero STITCH001 minori di 30: ", len(stitch002[stitch002.astype(float) < 30]))
print(
    "numero STITCH001 maggiori di 300: ", len(stitch002[stitch002.astype(float) > 300])
)

mask = df_esami_par_cal["codicestitch"] == "STITCH002"
df_esami_par_cal.loc[mask, "valore"] = df_esami_par_cal.loc[mask, "valore"].clip(
    30, 300
)

mask = df_esami_par_cal["codicestitch"] == "STITCH003"
df_esami_par_cal.loc[mask, "valore"] = df_esami_par_cal.loc[mask, "valore"].clip(
    60, 330
)

stitch002_dopo = df_esami_par_cal[df_esami_par_cal["codicestitch"] == "STITCH002"][
    "valore"
]

print("dopo update: ")
print(
    "numero STITCH001 minori di 30 dopo filtro: ",
    len(stitch002_dopo[stitch002_dopo < 30]),
)
print(
    "numero STITCH001 maggiori di 300 dopo filtro: ",
    len(stitch002_dopo[stitch002_dopo.astype(float) > 300]),
)

prima update: 
numero STITCH001 minori di 30:  4178
numero STITCH001 maggiori di 300:  42
dopo update: 
numero STITCH001 minori di 30 dopo filtro:  0
numero STITCH001 maggiori di 300 dopo filtro:  0


## Task 1.5

In [43]:
aa_prob_cuore_filtered_keys = (
    aa_prob_cuore[["idana", "idcentro"]]
    .drop_duplicates()
    .merge(
        groups_diagnosi_and_esami_keys,
        on=["idana", "idcentro"],
        how="inner",
    )
)

In [44]:
del groups_diagnosi_and_esami_keys

In [45]:
print(
    "numero pazienti inizio punto 5: ",
    len(aa_prob_cuore_filtered_keys[["idana", "idcentro"]].drop_duplicates()),
)

df_diagnosi_and_esami = df_diagnosi_and_esami.merge(
    aa_prob_cuore_filtered_keys,
    on=["idana", "idcentro"],
    how="inner",
)
print("df_diagnosi_and_esami merged")

numero pazienti inizio punto 5:  48955
df_diagnosi_and_esami merged


In [46]:
# TODO: a questo punto dato che per il punto 3 non abbiamo usato le prescrizioni, non dobbiamo usarle nemmeno qui le prescrizioni
#  in quanto no ritengo che siano eventi significativi, quindi qui vanno rivisti i filtri

# df_prescrizioni_diabete_farmaci = df_prescrizioni_diabete_farmaci.merge(
#     aa_prob_cuore_filtered_keys,
#     on=["idana", "idcentro"],
#     how="inner",
# )
# print("df_prescrizioni_diabete_farmaci merged")
# df_prescirizioni_non_diabete = df_prescirizioni_non_diabete.merge(
#     aa_prob_cuore_filtered_keys,
#     on=["idana", "idcentro"],
#     how="inner",
# )
# print("df_prescirizioni_non_diabete merged")
# df_prescrizioni_diabete_non_farmaci = df_prescrizioni_diabete_non_farmaci.merge(
#     aa_prob_cuore_filtered_keys,
#     on=["idana", "idcentro"],
#     how="inner",
# )
# print("df_prescrizioni_diabete_non_farmaci merged")

# df_diagnosi_and_esami_and_prescrioni = pd.concat(
#     objs=(
#         idf.set_index(["idana", "idcentro"])
#         for idf in [
#             df_diagnosi_and_esami[["idcentro", "idana", "data"]],
#             df_prescrizioni_diabete_farmaci[["idcentro", "idana", "data"]],
#             df_prescirizioni_non_diabete[["idcentro", "idana", "data"]],
#             df_prescrizioni_diabete_non_farmaci[["idcentro", "idana", "data"]],
#         ]
#     ),
#     join="inner",
# ).reset_index()


In [47]:
print("df_diagnosi_and_esami_and_prescrioni concatenated")
cont = (
    # df_diagnosi_and_esami_and_prescrioni[["idana", "idcentro"]]
    df_diagnosi_and_esami[["idana", "idcentro"]]
    .groupby(["idana", "idcentro"])
    .size()
    .reset_index(name="count")
)

print("paziente con minimo numero eventi", cont.sort_values(by=["count"]).head(1))
print("paziente con massimo numero eventi", cont.sort_values(by=["count"]).tail(1))
print("cont grouped")
cont_filtered = cont[cont["count"] >= 2]

select_all_events = df_diagnosi_and_esami.merge(
    # df_diagnosi_and_esami_and_prescrioni.merge(
    cont_filtered.reset_index()[["idana", "idcentro"]],
    on=["idana", "idcentro"],
    how="inner",
)

df_diagnosi_and_esami_and_prescrioni concatenated
paziente con minimo numero eventi        idana  idcentro  count
34102   6954       111      2
paziente con massimo numero eventi        idana  idcentro  count
21393   3566       116   2516
cont grouped


In [48]:
last_event = select_all_events.groupby(["idana", "idcentro"], group_keys=True)[
    "data"
].max()

In [49]:
print(
    "num pazienti in all_events: ",
    len(select_all_events[["idana", "idcentro"]].drop_duplicates()),
)

print(
    "df_problemi_cuore: ",
    len(aa_prob_cuore_filtered_keys[["idana", "idcentro"]].drop_duplicates()),
)

num pazienti in all_events:  48955
df_problemi_cuore:  48955


In [50]:
aa_prob_cuore_filtered = aa_prob_cuore_filtered_keys.merge(
    aa_prob_cuore[["idana", "idcentro", "data"]],
    on=["idana", "idcentro"],
    how="inner",
)

aa_prob_cuore_filtered["data"] = pd.to_datetime(
    aa_prob_cuore_filtered["data"], format="%Y-%m-%d"
)

last_problem = aa_prob_cuore_filtered.groupby(["idana", "idcentro"], group_keys=True)[
    "data"
].max()

In [51]:
del (
    aa_prob_cuore_filtered,
    cont,
    cont_filtered,
    # df_diagnosi_and_esami_and_prescrioni,
    aa_prob_cuore_filtered_keys,
    df_diagnosi_and_esami,
)

In [52]:

wanted_patient = select_all_events.join(
    (last_problem.ge(last_event - pd.DateOffset(months=6))).rename("label"),
    on=["idana", "idcentro"],
)

In [53]:
del last_problem, select_all_events, last_event

In [54]:
# TODO: delete wanted_patient with trajectory less than 6 months
# I don't understand here someting fishy is going on because I see only 127 days of difference
# between min and max date but the specification says 6 months
wanted_patient_6_months = wanted_patient.groupby(["idana", "idcentro"]).agg(
    {"data": ["min", "max"]}
)

In [55]:
wanted_patient_6_months["data_min"] = wanted_patient_6_months["data"]["min"]
wanted_patient_6_months["data_max"] = wanted_patient_6_months["data"]["max"]

wanted_patient_6_months["diff"] = (
    wanted_patient_6_months["data_max"] - wanted_patient_6_months["data_min"]
)

wanted_patient_6_months = wanted_patient_6_months[
    wanted_patient_6_months["diff"] >= pd.Timedelta("183 days")
]
wanted_patient_6_months = wanted_patient_6_months.sort_values(by=["diff"])
print("paziente traiettoria minima: ", wanted_patient_6_months.head(1))
print("paziente traiettoria massima: ", wanted_patient_6_months.tail(1))

wanted_patient_6_months_keys = (
    wanted_patient_6_months.stack()
    .reset_index()[["idana", "idcentro"]]
    .drop_duplicates()
)

paziente traiettoria minima:                       data              data_min   data_max     diff
                      min        max                               
idana idcentro                                                     
5078  61       2007-09-17 2008-03-18 2007-09-17 2008-03-18 183 days
paziente traiettoria massima:                       data              data_min   data_max       diff
                      min        max                                 
idana idcentro                                                       
3273  23       1900-01-05 2019-09-23 1900-01-05 2019-09-23 43725 days


In [56]:
print("RISULATI PUNTO 1.5")
# print(wanted_patient[["idana", "idcentro", "data", "label"]])
wanted_patient = wanted_patient.merge(
    wanted_patient_6_months_keys,
    on=["idana", "idcentro"],
    how="inner",
)

wanted_patient_keys = wanted_patient[["idana", "idcentro"]].drop_duplicates()

print(
    "pazienti fine punto 5: ",
    len(wanted_patient_keys),
)
wanted_patient1 = wanted_patient[wanted_patient["label"] == True]
unwanted_patient = wanted_patient[wanted_patient["label"] == False]
# print(wanted_patient1)
print("True rows patients: ", len(wanted_patient1))
print("False rows patients: ", len(unwanted_patient))
print("True patients: ", len(wanted_patient1[["idana", "idcentro"]].drop_duplicates()))
print(
    "False patients: ", len(unwanted_patient[["idana", "idcentro"]].drop_duplicates())
)

RISULATI PUNTO 1.5
pazienti fine punto 5:  47994
True rows patients:  4056694
False rows patients:  8237791
True patients:  16290
False patients:  31704


## Task 1.6
Some things for point 6 are done in point 2 and 3 to speed up computations


In [57]:
print("############## POINT 6 START ##############")

# TODO: controllare anagrafica e diagnosi
print("patients labels: ")
print(wanted_patient.isna().sum())
# qui ci sono 456 righe con data a nan

print("anagrafica: ")
df_anagrafica_attivi = df_anagrafica_attivi.merge(
    wanted_patient_keys, on=["idana", "idcentro"], how="inner"
)
print(df_anagrafica_attivi.isna().sum())

print("diagnosi: ")
df_diagnosi = df_diagnosi.merge(
    wanted_patient_keys, on=["idana", "idcentro"], how="inner"
)
print(df_diagnosi.isna().sum())
# qui ci sono 456 righe con data a nan
# qui ci sono 33k righe con valore a nan

df_esami_par = df_esami_par.merge(
    wanted_patient_keys, on=["idana", "idcentro"], how="inner"
)

print("esami lab parametri: ")
print(df_esami_par.isna().sum())
# qui ci sono 30k righe con valore a nan
# e 800k righe con anno primo accesso nan

df_esami_par_cal = df_esami_par_cal.merge(
    wanted_patient_keys, on=["idana", "idcentro"], how="inner"
)
print("esami lab parametri calcolati: ")
print(df_esami_par_cal.isna().sum())
# qui ci sono 900k righe con codice amd nan
# e 300k righe con anno primo accesso nan

print(df_esami_par_cal.groupby(["codiceamd"]).size())
print(df_esami_par_cal.groupby(["codicestitch"]).size())
print(
    df_esami_par_cal[df_esami_par_cal["codiceamd"].isna()]["codicestitch"]
    .isin(["STITCH003", "STITCH004"])
    .sum()
)

df_esami_stru = df_esami_stru.merge(
    wanted_patient_keys, on=["idana", "idcentro"], how="inner"
)
print("esami strumentali: ")
print(df_esami_stru.isna().sum())
# qui ci sono 20k righe con valore a nan
# e 20k righe con anno primo accesso nan

print(df_esami_stru.groupby(["codiceamd"]).size())
# alcuni codici amd sono presenti in proporzioni molto maggiori rispetto ad altri

print("prescrizioni diabete farmaci: ")
df_prescrizioni_diabete_farmaci = df_prescrizioni_diabete_farmaci.merge(
    wanted_patient_keys, on=["idana", "idcentro"], how="inner"
)
print(df_prescrizioni_diabete_farmaci.isna().sum())
# qui ci sono 38 righe con codice atc nan da gestire
# e 250k righe con anno primo accesso nan
print(df_prescrizioni_diabete_farmaci.groupby(["codiceatc"]).size())

df_prescrizioni_diabete_non_farmaci = df_prescrizioni_diabete_non_farmaci.merge(
    wanted_patient_keys, on=["idana", "idcentro"], how="inner"
)

print(df_prescrizioni_diabete_non_farmaci.groupby(["codiceamd"]).size())
# qui abbiamo un codice amd096 che è presente in sole 32 righe e quindi completamente
# sbilanciato rispetto agli altri codici amd presenti in grandi quantità, quindi lo scarterei
# poi due codici amd086 e amd152 riportano la stessa descrizione e quindi li unirei in un unico codice,
# l'unico problema è che a quel punto la maggioranza dei codici sarebbero l'unione di questo 120k
# e i rimanenti 25k sarebbero altri due codici, quindi non so se sia il caso di unirli
print("prescrizioni diabete non farmaci: ")
print(df_prescrizioni_diabete_non_farmaci.isna().sum())
# qui ci sono 15k righe con valore nan
# e 15k righe con anno primo accesso nan
print(df_prescrizioni_diabete_non_farmaci.groupby(["codiceamd"]).size())

df_prescirizioni_non_diabete = df_prescirizioni_non_diabete.merge(
    wanted_patient_keys, on=["idana", "idcentro"], how="inner"
)
print("prescrizioni non diabete: ")
print("no nan")
#print(df_prescirizioni_non_diabete.isna().sum())
# qui ci sono 250k righe con anno primo accesso nan

esami lab parametri: 
idcentro                  0
idana                     0
data                      0
codiceamd                 0
valore                30385
annonascita               0
annoprimoaccesso     800937
annodecesso         6508443
dtype: int64
esami lab parametri calcolati: 
idcentro                  0
idana                     0
data                      0
codiceamd            898091
valore                    0
codicestitch              0
annonascita               0
annoprimoaccesso     309337
annodecesso         2441575
dtype: int64
codiceamd
AMD013    345322
AMD304    535549
AMD927    974438
dtype: int64
codicestitch
STITCH001    974438
STITCH002    345322
STITCH003    362542
STITCH004    535549
STITCH005    535549
dtype: int64
898091
esami strumentali: 
idcentro                 0
idana                    0
data                     0
codiceamd                0
valore               21143
annonascita              0
annoprimoaccesso     21219
annodecesso         261086
d

In [ ]:
print(wanted_patient.head())

In [58]:
aa_prob_cuore.head()


,idcentro,idana,sesso,annodiagnosidiabete,scolarita,statocivile,professione,origine,annonascita,annoprimoaccesso,annodecesso,data,codiceamd,valore
0,100,3080,M,1988-01-01,NaN,NaN,NaN,NaN,1936-01-01,1991-01-01,NaT,2005-02-21,AMD049,S
1,100,3080,M,1988-01-01,NaN,NaN,NaN,NaN,1936-01-01,1991-01-01,NaT,2007-09-12,AMD049,S
2,100,3080,M,1988-01-01,NaN,NaN,NaN,NaN,1936-01-01,1991-01-01,NaT,2008-04-03,AMD049,S
3,100,3080,M,1988-01-01,NaN,NaN,NaN,NaN,1936-01-01,1991-01-01,NaT,2009-03-20,AMD049,S
4,100,3080,M,1988-01-01,NaN,NaN,NaN,NaN,1936-01-01,1991-01-01,NaT,2015-09-08,AMD049,S


In [59]:
aa_prob_cuore.isna().sum()

# We have a low of nan values in the colums scolarita statocivile, professione and expecially origine (maybe we can drop them)

idcentro                    0
idana                       0
sesso                       0
annodiagnosidiabete         0
scolarita              150368
statocivile            106383
professione            110947
origine                229770
annonascita                 0
annoprimoaccesso        25132
annodecesso            207224
data                        0
codiceamd                   0
valore                      0
dtype: int64

In [60]:
aa_prob_cuore.describe()

,idcentro,idana,scolarita,statocivile,professione,origine
count,232713.000000,2.327130e+05,82345.000000,126330.000000,121766.000000,2943.000000
mean,111.880750,5.401526e+03,2.194681,2.126455,7.400202,1.074074
std,70.189405,1.960398e+04,0.546911,0.553518,2.804705,0.416294
min,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000
25%,60.000000,1.652000e+03,2.000000,2.000000,7.000000,1.000000
50%,111.000000,3.829000e+03,2.000000,2.000000,9.000000,1.000000
75%,163.000000,7.360000e+03,2.000000,2.000000,9.000000,1.000000
max,500.000000,8.692111e+06,5.000000,6.000000,10.000000,5.000000


Show the multiplicity of values in aa_prob_cuore (from anagraficapazientiattivi.csv) of scolarita, statocivile, professione and origine

In [61]:
aa_prob_cuore.scolarita.value_counts()

2.0    63229
3.0    13260
1.0     2981
4.0     2873
5.0        2
Name: scolarita, dtype: int64

In [62]:
#aa_prob_cuore.statocivile.value_counts()

2.0    99833
3.0    14129
1.0     7631
4.0     4735
5.0        1
6.0        1
Name: statocivile, dtype: int64

In [63]:
#aa_prob_cuore.professione.value_counts()

9.0     78980
4.0      9178
1.0      8999
7.0      6254
2.0      6219
10.0     5422
3.0      2915
6.0      2429
8.0      1319
5.0        51
Name: professione, dtype: int64

In [64]:
aa_prob_cuore.origine.value_counts()

1.0    2843
3.0      82
5.0      12
2.0       6
Name: origine, dtype: int64

In [67]:
aa_prob_cuore.drop(columns=['scolarita', 'statocivile', 'professione', 'origine'], inplace=True)

aa_prob_cuore.describe()

,idcentro,idana
count,232713.000000,2.327130e+05
mean,111.880750,5.401526e+03
std,70.189405,1.960398e+04
min,1.000000,1.000000e+00
25%,60.000000,1.652000e+03
50%,111.000000,3.829000e+03
75%,163.000000,7.360000e+03
max,500.000000,8.692111e+06
